In [0]:
%sql
with 
subscriptions as (
    select hs_user_id,
           dt subscription_start_date,
           free_trial_start_date,
           free_trial_end_date,
           paid_subscription_start_date,
           paid_subscription_end_date,
           lead(dt) over(partition by hs_user_id order by dt) next_subscription_start_date,
           lead(paid_subscription_start_date) over(partition by hs_user_id order by dt) next_paid_subscription_start_date,
           lead(paid_subscription_end_date) over(partition by hs_user_id order by dt) next_paid_subscription_end_date
    from silver.fact_subscription
),
canvas as (
    select id canvas_id,
           name,
           created_at::date created_date,
           first_entry::date first_entry_date,
           last_entry::date last_entry_date
    from bronze.braze_canvas
    where lower(name) like '%winback%'
),
canvas_entries as (
    select s.hs_user_id,
           e.dt canvas_entry_date,
           e.canvas_name,
           e.canvas_id,
           e.canvas_variation_name,
           c.created_date canvas_created_date,
           s.subscription_start_date,
           s.free_trial_start_date,
           s.free_trial_end_date,
           s.paid_subscription_start_date,
           s.paid_subscription_end_date,
           s.next_subscription_start_date,
           s.next_paid_subscription_start_date,
           s.next_paid_subscription_end_date
    from bronze.braze_events e
    inner join canvas c
      on e.canvas_id = c.canvas_id
    inner join subscriptions s
      on s.hs_user_id = e.external_user_id
      and e.dt between s.subscription_start_date and coalesce(s.next_subscription_start_date,'9999-12-31')
    where e.event_name = 'users_canvas_entry'
      --and e.dt between '2021-01-01' and '2021-12-31'
)
select c.canvas_name,
       c.canvas_created_date,
       count(case when lower(c.canvas_variation_name) <> 'control' and event_name = 'users_canvas_entry' then 1 else null end) treatment_canvas_entries,
       count(case when lower(c.canvas_variation_name) <> 'control' and event_name = 'users_messages_email_delivery' then 1 else null end) treatment_emails_delivered,
       count(case when lower(c.canvas_variation_name) <> 'control' and event_name = 'users_messages_email_open' then 1 else null end) treatment_emails_opened,
       count(case when lower(c.canvas_variation_name) <> 'control' and event_name = 'users_messages_email_click' then 1 else null end) treatment_emails_clicked,
       count(case when lower(c.canvas_variation_name) <> 'control' and event_name = 'users_messages_email_unsubscribe' then 1 else null end) treatment_emails_unsubscribed,
       count(case when lower(c.canvas_variation_name) <> 'control' and event_name = 'users_messages_email_mark_as_spam' then 1 else null end) treatment_emails_marked_as_spam,
       count(case when lower(c.canvas_variation_name) <> 'control' and event_name = 'users_messages_webhook_send' then 1 else null end) treatment_webhook_sent,
       count(case when lower(c.canvas_variation_name) <> 'control' and event_name = 'users_canvas_conversion' then 1 else null end) treatment_canvas_conversion,
       count(distinct case when lower(c.canvas_variation_name) <> 'control' and datediff(coalesce(next_paid_subscription_start_date, '9999-12-31'), canvas_entry_date) <= 28 then c.hs_user_id else null end) treatment_conversions,
       count(case when lower(c.canvas_variation_name) = 'control' and event_name = 'users_canvas_entry' then 1 else null end) control_canvas_entries,
       count(case when lower(c.canvas_variation_name) = 'control' and event_name = 'users_messages_email_delivery' then 1 else null end) control_emails_delivered,
       count(case when lower(c.canvas_variation_name) = 'control' and event_name = 'users_messages_email_open' then 1 else null end) control_emails_opened,
       count(case when lower(c.canvas_variation_name) = 'control' and event_name = 'users_messages_email_click' then 1 else null end) control_emails_clicked,
       count(case when lower(c.canvas_variation_name) = 'control' and event_name = 'users_messages_email_unsubscribe' then 1 else null end) control_emails_unsubscribed,
       count(case when lower(c.canvas_variation_name) = 'control' and event_name = 'users_messages_email_mark_as_spam' then 1 else null end) control_emails_marked_as_spam,
       count(case when lower(c.canvas_variation_name) = 'control' and event_name = 'users_messages_webhook_send' then 1 else null end) control_webhook_sent,
       count(case when lower(c.canvas_variation_name) = 'control' and event_name = 'users_canvas_conversion' then 1 else null end) control_canvas_conversion,
       count(distinct case when lower(c.canvas_variation_name) = 'control' and datediff(coalesce(next_paid_subscription_start_date, '9999-12-31'), canvas_entry_date) <= 28 then c.hs_user_id else null end) control_conversions
from canvas_entries c
inner join bronze.braze_events e
  on c.hs_user_id = e.external_user_id
  and c.canvas_id = e.canvas_id
group by 1,2
order by 2 desc